### 풍자 문장 생성

In [1]:
# 토큰 불러오기
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("sarcasm_generate_0526")

LangSmith 추적을 시작합니다.
[프로젝트명]
sarcasm_generate_0526


In [3]:
# llm 모델 정의
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=1.0, model_name="gpt-4o")

In [ ]:
# prompt 가져오기
from langchain_core.prompts import load_prompt

prompt = load_prompt(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/generate_sarcasm_zeroshot_jeonghoon.yaml",
    encoding="utf-8",
)
prompt

PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='당신은 리뷰 문장의 맥락과 감정에서 사용자의 숨은 의도를 분석하고, 같은 의도를 가진 두 가지 버전의 문장을 생성하는 언어 전문가입니다.\n\n주어진 context를 읽고 다음 단계를 따라 사고를 전개하세요:\n1. context의 주요 사건이나 상황을 명확하게 한 문장으로 요약하세요.\n2. context에서 화자들이 느낄 수 있는 감정을 2~3가지 키워드로 정리하세요.\n3. 위 감정과 상황을 반영하여 해당 상황에서 화자가 말할 만한 sarcasm 문장을 작성하세요.\n   - sarcasm 문장은 반어적이며, 약간의 유머를 담되 과장하거나 현실과 모순된 표현을 반드시 포함하세요.\n4. 동일한 감정과 상황을 기반으로 sarcasm이 아닌 자연스러운 non_sarcasm 문장을 작성하세요.\n5. 각 문장은 자연스러운 한국어로 작성하세요.\n\n출력 형식은 아래와 같이 JSON으로 작성하세요:\n\n{{\n  "sarcasm": "string",\n  "non_sarcasm": "string"\n}}\n\n\ncontext: {context}\n\n#Answer:\n')

In [5]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Optional


# ① 원하는 데이터 구조 정의
class SarcasmOutput(BaseModel):
    sarcasm: str = Field(description="주어진 context를 반어적이고 과장하여 표현한 문장")
    non_sarcasm: str = Field(description="주어진 context를 non_sarcasm으로 표현한 문장")


# ② 출력 파서 설정
output_parser = PydanticOutputParser(pydantic_object=SarcasmOutput)

# ③ 출력 포맷 지시사항 확인
print(output_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sarcasm": {"description": "주어진 context를 반어적이고 과장하여 표현한 문장", "title": "Sarcasm", "type": "string"}, "non_sarcasm": {"description": "주어진 context를 non_sarcasm으로 표현한 문장", "title": "Non Sarcasm", "type": "string"}}, "required": ["sarcasm", "non_sarcasm"]}
```


In [6]:
format_instructions = output_parser.get_format_instructions()
prompt = prompt.partial(format_instructions=format_instructions)

In [7]:
# 체인 생성
chain = prompt | llm | output_parser

In [9]:
# 데이터 불러오기
import pandas as pd

df = pd.read_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/negative_data_0531.csv",
    encoding="utf-8",
)

# df = df[300:639].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Column1       341 non-null    int64 
 1   subject       341 non-null    object
 2   speaker_type  341 non-null    object
 3   context       341 non-null    object
 4   sentiment     341 non-null    object
dtypes: int64(1), object(4)
memory usage: 13.4+ KB


In [10]:
# input 데이터 로드
input_data = df["context"]

input_data_list = input_data.to_list()

In [11]:
results = []

for context in input_data_list:
    try:
        # LLM 호출 및 결과 반환
        result = chain.invoke(
            {"context": context, "format_instructions": format_instructions}
        )

        # 결과가 dict 또는 SarcasmOutput 객체라면 속성 접근
        sarcasm = getattr(result, "sarcasm", None)
        non_sarcasm = getattr(result, "non_sarcasm", None)

    except Exception as e:
        print(f"[ERROR] context 처리 중 오류 발생:\n{context}\n{e}")
        sarcasm = None
        non_sarcasm = None

    results.append(
        {
            "context": context,
            "sarcasm": sarcasm,
            "non_sarcasm": non_sarcasm,
            "explanation": None,
        }
    )

# DataFrame 생성 및 저장
test_df = pd.DataFrame(results)

test_df.head()

test_df.to_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/generate_sarcasm/341개 데이터_0531.csv",
    encoding="utf-8-sig",
)


Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=8039782c-16ae-4c66-adbc-dd12931888c5,id=8039782c-16ae-4c66-adbc-dd12931888c5; trace=8039782c-16ae-4c66-adbc-dd12931888c5,id=87547232-c5b3-4748-b5e7-bf9ab3f19a71; trace=8039782c-16ae-4c66-adbc-dd12931888c5,id=1eeb6aca-9c21-42a5-8365-c5df818d8491
Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')
Failed to send comp

[ERROR] context 처리 중 오류 발생:
가을 단풍과 연말 계획에 대한 대화로, 시간의 흐름에 대한 아쉬움을 나누는 상황
Failed to parse SarcasmOutput from completion 1. Got: 1 validation error for SarcasmOutput
  Input should be a valid dictionary or instance of SarcasmOutput [type=model_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipar

Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipar

In [118]:
import pandas as pd

test_df = pd.DataFrame(results)

test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   context      100 non-null    object
 1   sarcasm      100 non-null    object
 2   non_sarcasm  100 non-null    object
 3   explanation  0 non-null      object
dtypes: object(4)
memory usage: 3.3+ KB


In [ ]:
test_df.to_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/generate_sarcasm/1000개_temperature_1.0_zeroshot_1.csv",
    encoding="utf-8-sig",
)

### 칼럼명 통일

In [12]:
import pandas as pd

df = pd.read_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/generate_sarcasm/341개 데이터_0531.csv",
    encoding="utf-8-sig",
)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   341 non-null    int64  
 1   context      341 non-null    object 
 2   sarcasm      340 non-null    object 
 3   non_sarcasm  340 non-null    object 
 4   explanation  0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 13.4+ KB


In [13]:
# sarcasm 데이터 가공
df_sarcasm = df[["context", "sarcasm"]].copy()
df_sarcasm.rename(columns={"sarcasm": "response"}, inplace=True)
df_sarcasm["label"] = "Sarcasm"
df_sarcasm["sarcasm_explanation"] = None

# non-sarcasm 데이터 가공
df_non_sarcasm = df[["context", "non_sarcasm"]].copy()
df_non_sarcasm.rename(columns={"non_sarcasm": "response"}, inplace=True)
df_non_sarcasm["label"] = "Non-sarcasm"
df_non_sarcasm["sarcasm_explanation"] = None

# 병합 후 인덱스 재설정
df_final = pd.concat([df_sarcasm, df_non_sarcasm], ignore_index=True)
df_final.reset_index(inplace=True)

# 결과 출력 예시
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   index                682 non-null    int64 
 1   context              682 non-null    object
 2   response             680 non-null    object
 3   label                682 non-null    object
 4   sarcasm_explanation  0 non-null      object
dtypes: int64(1), object(4)
memory usage: 26.8+ KB


In [14]:
# 결과 저장 (선택)
df_final.to_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/generate_sarcasm/341개 데이터_전처리_0531.csv",
    index=False,
)

### sarcasm, non_sarcasm explanation 작성

In [15]:
# 토큰 불러오기
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [16]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("sarcasm_generate_0526")

LangSmith 추적을 시작합니다.
[프로젝트명]
sarcasm_generate_0526


In [17]:
# llm 모델 정의
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=1.0, model_name="gpt-4o-mini")

In [18]:
# 데이터 불러오기
import pandas as pd

df = pd.read_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/generate_sarcasm/341개 데이터_전처리_0531.csv",
    encoding="utf-8",
)

df = df.dropna(subset=['context'])
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                682 non-null    int64  
 1   context              682 non-null    object 
 2   response             680 non-null    object 
 3   label                682 non-null    object 
 4   sarcasm_explanation  0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 26.8+ KB


In [19]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "response", "label"],
    template="""
당신은 sarcasm과 non_sarcasm을 구분하는 언어 전문가입니다.
다음 내용을 참고하여, response 문장이 '{label}'인 이유를 한 문장으로 작성해주세요.

context: {context}
response: {response}
label: {label}

#Answer:
""",
)

In [20]:
from tqdm import tqdm

explanations = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    input_data = {
        "context": row["context"],
        "response": row["response"],
        "label": row["label"],
    }
    answer = llm.invoke(prompt.format(**input_data)).content.strip()
    explanations.append(answer)

df["sarcasm_explanation"] = explanations

100%|██████████| 682/682 [17:20<00:00,  1.53s/it]


In [21]:
df.head()

,index,context,response,label,sarcasm_explanation
0,0,결혼과 연애에 대한 개인의 생각과 고민을 나누는 대화,"아, 결혼은 모든 문제를 마법처럼 해결해 주는 해답이죠! 모든 게 그냥 완벽해질 거예요.",Sarcasm,이 문장은 결혼이 실제로 문제를 해결해 주지 않음에도 불구하고 그 반대의 의미로 과...
1,1,동네 강아지 미용실이 없어져서 반려견의 미용과 겨울철 관리에 대한 우려를 나눈 대화,이제 우리 강아지를 미용하려면 직접 가위를 잡아야겠네요. 미용 기술 배우는 건 시간...,Sarcasm,이 문장은 강아지를 직접 미용해야 한다는 불만을 경 sarcastically 표현하...
2,2,운동 중 부상 경험과 그로 인한 고통에 대해 이야기하는 대화,"아, 운동은 정말 최고의 스트레스 해소법이죠. 이 통증 가득한 몸도 덤으로 얻고요!",Sarcasm,이 문장은 운동의 이점을 언급하면서 통증을 덤으로 얻는다고 말해 조롱하는 듯한 어조...
3,3,피부과 시술과 다이어트에 대한 고민과 경험을 나누는 대화,"아, 정말 피부과 시술과 다이어트는 돈과 시간을 남아도는 사람들만 할 수 있는 여유...",Sarcasm,이 문장은 피부과 시술과 다이어트에 대한 어려움을 조롱하며 이런 시술들이 실제로는 ...
4,4,"엄마의 재택근무와 급여 감소, 코로나19 상황에서의 일자리 문제에 대한 대화","아, 재택근무 덕분에 월급도 줄고 집에 계속 머물 수 있어서 좋네요. 마치 휴가처럼요!",Sarcasm,"이 문장은 재택근무와 급여 감소로 인한 어려움을 오히려 긍정적으로 표현하며, 상황의..."


In [22]:
df.to_csv(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/generate_sarcasm/0531_최종.csv"
)